<font style='font-size:1.5em'>**🧑‍🏫 Week 08 Lecture**</font><br>
<font style='font-size:1.3em;color:#888888'>NOTEBOOK 02: Normalising and storing data to a database</font>

<font style='font-size:1.2em;color:#e26a4f;font-weight:bold'>LSE DS105A – Data for Data Science (2024/25) </font>



<div style="color: #333333; background-color:rgba(226, 106, 79, 0.075); border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); padding: 20px 0 20px 10px; margin: 10px 0 10px 0; flex: 1 1 calc(45% - 20px);min-width: 250px;max-width: 350px;align-items:top;min-height: calc(45% - 20px); box-sizing: border-box;font-size:0.9em;">

🗓️ **DATE:** 21 November 2024 

⌚ **TIME:** 16.00-18.00

📍 **LOCATION:** CLM.5.02
</div>


**AUTHORS:**  Dr. [Jon Cardoso-Silva](https://jonjoncardoso.github.io)

**DEPARTMENT:** [LSE Data Science Institute](https://lse.ac.uk/dsi)

**OBJECTIVE**: In Weeks 08 and 09, we will revisit the data science workflow (collection -> storage -> processing -> analysis -> visualization) but with new tools and techniques. We will cover how to use APIs that require authentication to collect data, we will revisit the notion of API endpoints and then, once we have collected the data, we will learn how to store it in a more structured way using databases.

<details style="width:70%;font-size:1em;border: 1px solid #aaa;border-radius: 4px;padding: .5em;margin-left:0em"><summary style="font-weight:bold">🖇️ EXPAND FOR USEFUL LINKS</summary>

- Python 3's [`venv` module documentation](https://docs.python.org/3/library/venv.html)

- W3 Schools' [HTTP Request Methods](https://www.w3schools.com/tags/ref_httpmethods.asp) page

- [Reddit API documentation](https://www.reddit.com/dev/api/)
- [Reddit API Rules](https://support.reddithelp.com/hc/en-us/articles/16160319875092-Reddit-Data-API-Wiki)

- The [JSON Crack Extension](https://marketplace.visualstudio.com/items?itemName=AykutSarac.jsoncrack-vscode) for VS Code to visually inspect JSON files.

- 🐼 pandas' [`pd.json_normalize()` function documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html)

- [The `pydotenv` library](https://pypi.org/project/python-dotenv/)
- [What is the gitignore file?](https://www.atlassian.com/git/tutorials/saving-changes/gitignore)


Not covered here but relevant to your upcoming assignment:

- [Spotify API documentation](https://developer.spotify.com/documentation/web-api/)
- [Spotify Getting Started Guide](https://developer.spotify.com/documentation/web-api/quick-start/)

</details>


---

**⚙️ SETUP**

Before you continue, set up your Python environment. Check the instructions under the ['🐍 Python environment' section on README](../README.md#🐍-python-environment).

In [1]:
import os
import json
import requests

import pandas as pd

from sqlalchemy import create_engine, text

from datetime import datetime

from IPython.display import Image

# 1. Read the JSON files

At the time of writing this notebook, we have collected the top 1000 posts from the subreddit `r/datascience` and saved them in a single JSON file. 

Our goal now is to store this data in a database. But before we do that, let's read the JSON file and inspect its contents.

In [2]:
filepath = '../data/reddit/raw/datascience_top1000_2024_11_21_09_56.json'

# How does the data look like?
pd.read_json(filepath).head()

,kind,data
0,t3,"{'approved_at_utc': None, 'subreddit': 'datasc..."
1,t3,"{'approved_at_utc': None, 'subreddit': 'datasc..."
2,t3,"{'approved_at_utc': None, 'subreddit': 'datasc..."
3,t3,"{'approved_at_utc': None, 'subreddit': 'datasc..."
4,t3,"{'approved_at_utc': None, 'subreddit': 'datasc..."


Using my knowledge of Week 07, I know that there is a `data` key in the JSON file that contains a list of dictionaries. Each dictionary represents a post in the subreddit. I can use the `pd.json_normalize()` function to flatten this list of dictionaries into a DataFrame.

In [3]:
df = pd.json_normalize(pd.read_json(filepath)['data'])

print(f"Number of posts: {df.shape[0]} {'(🎉 AS EXPECTED)' if df.shape[0] == 1000 else ''}")

Number of posts: 1000 (🎉 AS EXPECTED)


# 2 Inspecting the DataFrame (to study later)

In this SECTION, I demonstrate how I would inspect this unfamiliar DataFrame to understand its structure and decide which columns to keep.

First of all, my interest with the current endpoint lies in understanding the type of content that is heavily favoured in the `r/datascience` subreddit. I might also want to look at, say, the comments later but for now, I will focus on the posts themselves. Let's check what each of these columns represent and decide which ones we want to keep.

In [4]:
# Oh no, there are 870 columns!
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 870 entries, approved_at_utc to media_metadata.8u50wr1mo7ic1.id
dtypes: bool(28), float64(199), int64(10), object(633)
memory usage: 6.5+ MB


In [5]:
# There are just too many!
df.columns

Index(['approved_at_utc', 'subreddit', 'selftext', 'author_fullname', 'saved',
       'mod_reason_title', 'gilded', 'clicked', 'title', 'link_flair_richtext',
       ...
       'media.oembed.description', 'media.oembed.mean_alpha',
       'media_metadata.8u50wr1mo7ic1.status', 'media_metadata.8u50wr1mo7ic1.e',
       'media_metadata.8u50wr1mo7ic1.m', 'media_metadata.8u50wr1mo7ic1.p',
       'media_metadata.8u50wr1mo7ic1.s.y', 'media_metadata.8u50wr1mo7ic1.s.x',
       'media_metadata.8u50wr1mo7ic1.s.u', 'media_metadata.8u50wr1mo7ic1.id'],
      dtype='object', length=870)

## Investigate columns that were automatically normalised

All the columns that have a `.` in their name were automatically normalised by `pd.json_normalize()`. 

Some had deeper nested structures, producing more `.` in their names.

In [6]:
# Loop over all column names and identify those with `.` in them

normalized_cols = [col for col in df.columns if "." in col]
print(f"There are {len(normalized_cols)} columns with `.` in their name.")

There are 762 columns with `.` in their name.


Right, so A LOT of columns start with `data.`. Let's rename them to make it easier to work with:

## Remove columns by prefix

What are the prevalent prefixes in the column names?

In [7]:
prefixes = pd.Series([col.split(".")[0] for col in normalized_cols if "." in col]).value_counts()
prefixes

media_metadata        718
media                  16
secure_media           16
secure_media_embed      5
media_embed             4
preview                 2
gallery_data            1
Name: count, dtype: int64

I don't think I care about any of those! Let's drop them all.

In [8]:
prefixes.index

Index(['media_metadata', 'media', 'secure_media', 'secure_media_embed',
       'media_embed', 'preview', 'gallery_data'],
      dtype='object')

💡 **Now here's a pro-tip!** If you want to check if a string starts with a prefix you can do:

```python
my_string_variable.startswith('prefix')
```

To check if the string contains any of the prefixes in a list, you can do:

```python
any([my_string_variable.startswith(prefix) for prefix in my_prefixes_list])
```

But there is a nicer way to do that! You can simply pass a [tuple](https://www.w3schools.com/python/python_tuples.asp) of prefixes to the `str.startswith()` method:

```python
my_string_variable.startswith(('prefix1', 'prefix2', 'prefix3'))
```

In [9]:
irrelevant_prefixes = ('secure_media', 'media', 'preview', 'gallery_data')
columns_to_drop = [col for col in df.columns if col.startswith(irrelevant_prefixes)]

df = df.drop(columns=columns_to_drop)

In [10]:
# We got rid of a bunch of columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 105 entries, approved_at_utc to author_cakeday
dtypes: bool(27), float64(5), int64(10), object(63)
memory usage: 635.9+ KB


## Identify columns that have weird data types (or are constant)

For each column, check if it is made up primarily of lists (Return the string 'list' if it is).

If not, check if it has just one single value (Return the string 'single' if it is). Otherwise, return the string 'multiple'.

In [11]:
def get_column_type(column):
    """
    Determine the type of a column based on its values.

    Parameters:
        column (pandas.Series): The column to analyze.

    Returns:
        str: The type of the column. Possible values are 'list', 'single', or 'multiple'.
    """

    if any(isinstance(i, list) for i in column):
        return 'list'
    elif column.nunique() == 1:
        return 'single'
    else:
        return 'multiple'

# Let's see what types of columns we have
survey_column_types = (
    df.apply(get_column_type)
      .reset_index(name='column_type')
      .rename(columns={'index': 'column'})
      .groupby(['column_type'])
      .apply(lambda x: pd.Series({'count': len(x), 'columns': x['column'].tolist()}),
             include_groups=False)
)

# After inspecting, I decided that I only care about the columns that have multiple values
survey_column_types

,count,columns
column_type,,
list,8,"[link_flair_richtext, user_reports, author_fla..."
multiple,59,"[approved_at_utc, selftext, author_fullname, m..."
single,38,"[subreddit, saved, gilded, clicked, subreddit_..."


After some manual investigation, I decided to keep only the columns that have multiple values.

In [12]:
multiple_value_columns = survey_column_types.loc['multiple', 'columns']

# We use copy to avoid SettingDWithCopyWarning later on
df = df[multiple_value_columns].copy()

## It should be easier to select for what we want now

After further inspection of the remaining columns, I decided to keep only the following columns:

In [13]:
selected_columns =  [
    'id', 'title', 'permalink', 'post_hint',  'url', 'created_utc',           # Identifiers 
    'ups', 'upvote_ratio', 'score',                                           # Votes
    'num_comments', 'is_original_content', 'is_self',                         # Post metadata
    'author', 'author_fullname',                                              # Author metadata
    'content_categories', 'edited', 'domain'                                  # Misc metadata
]

In [14]:
# We use copy to avoid SettingDWithCopyWarning later on
df = df[selected_columns].copy()

## 2.3 Cleaning up the DataFrame

After A LOT OF WORK, I can summarise my entire initial cleaning of the DataFrame in the following steps:

In [15]:
# Points to where the data is stored
filepath = '../data/reddit/raw/datascience_top1000_2024_11_21_09_56.json'


# Columns that I identified as relevant
selected_columns =     [
    'id', 'title', 'permalink', 'post_hint',  'url', 'created_utc',            # Identifiers 
     'ups', 'upvote_ratio', 'score',                                           # Votes
     'num_comments', 'is_original_content', 'is_self',                         # Post metadata
     'author', 'author_fullname',                                              # Author metadata
     'content_categories', 'edited', 'domain']                                 # Misc metadata

df = (
    pd.json_normalize(pd.read_json(filepath)['data'])

    # Reformat the `created_utc` column to a human-readable format
    .assign(created_utc = lambda x: x['created_utc'].apply(lambda x: datetime.fromtimestamp(x)),
            permalink = lambda x: x['permalink'].apply(lambda x: f"https://www.reddit.com{x}"))

    # Select only the columns that I identified as relevant
    .loc[:, selected_columns]
)

# The top 5 posts
# Set an index just to make it easier for me to click on the links
df.head(5).set_index('permalink')

,id,title,post_hint,url,created_utc,ups,upvote_ratio,score,num_comments,is_original_content,is_self,author,author_fullname,content_categories,edited,domain
permalink,,,,,,,,,,,,,,,,
https://www.reddit.com/r/datascience/comments/1f6ztk4/how_to_avoid_12assed_data_analysis/,1f6ztk4,How to avoid 1/2-assed data analysis,image,https://i.redd.it/l6zd3zsa9cmd1.jpeg,2024-09-02 07:18:04,3122,0.98,3122,56,False,False,BdR76,t2_sejrw,None,False,i.redd.it
https://www.reddit.com/r/datascience/comments/1feyx6h/favourite_piece_of_code/,1feyx6h,Favourite piece of code 🤣,image,https://i.redd.it/jqvbe1ycrcod1.jpeg,2024-09-12 11:06:40,2810,0.98,2810,102,False,False,nobody_undefined,t2_dh9o0mnfv,None,False,i.redd.it
https://www.reddit.com/r/datascience/comments/1dsnbww/youre_not_helping_excel_please_stop_helping/,1dsnbww,"You're not helping, Excel! please STOP HELPING!!!",image,https://i.redd.it/ccxo0ajs3v9d1.jpeg,2024-07-01 08:46:27,1808,0.97,1808,155,False,False,BdR76,t2_sejrw,None,False,i.redd.it
https://www.reddit.com/r/datascience/comments/1afgvv9/friendly_reminder_not_to_work_too_hard_youll_just/,1afgvv9,Friendly reminder not to work too hard. You'll...,NaN,https://www.reddit.com/r/datascience/comments/...,2024-01-31 12:37:25,1704,0.93,1704,204,False,True,[deleted],NaN,None,False,self.datascience
https://www.reddit.com/r/datascience/comments/1b1au2f/data_scientist_quits_her_job_at_spotify/,1b1au2f,Data scientist quits her job at Spotify,rich:video,https://youtu.be/OMI4Wu9wnY0?si=teFkXgTnPmUAuAyU,2024-02-27 12:37:07,1398,0.91,1398,373,False,False,Direct-Touch469,t2_uy28jztl,None,False,youtu.be


# 3. Moving this to a database

We've learned about CSV, JSON, and plain text (.txt) files. These are all valid ways to store data, but if you see your data growing and need access quickly, you might consider using a **database**.


## 3.1 What is a database?

As this is not a database course, I will focus only on the basics. We will use a simple database called **SQLite**. It is good for us because it is simple and built into Python.

SQLite is:

- self-contained, 
- serverless,
- zero-configuration,
- transactional SQL database engine. 

As usual, there is a good W3Schools tutorial on [SQLite](https://www.w3schools.com/sql/sql_intro.asp).

<div style="width:70%;border: 1px solid #aaa; border-radius:1em; padding: 1em; margin: 1em 0;">


🌰 **In a nutshell: what is a relational (SQL) database?**

- Think of it as an even more structured way to store tables (data frames).

- If you have a huge table, it is a much more efficient way to store and query data.

- Merges are much faster than in 🐼 `pandas` if you have a lot of data.

- Every **tuple** (nearly the same concept of a row) in a table is uniquely identified by a **primary key**, a column, or a set of columns that uniquely identifies a row.

- Tables can be related to each other using **foreign keys**. This is a way to link tables together.

- SQL stands for **Structured Query Language**. It is a language used to interact with databases. You can use it to create, read, update, and delete data from a database.

- Most of 🐼 `pandas` functions are essentially an adaptation of SQL

- SQLite, the database we will use, is a simple database that comes installed with Python.

- ⭐ SQLite can store multiple tables in a single file.

- A SQLite database file can grow to be as large as about 281 terabytes

</div>

## 3.2 Connecting and exporting the DataFrame to a SQLite database

<font style="color:#e26a4f;font-weight:bold;">**THIS IS USEFUL TO KNOW BUT NOT THE WAY WE WANT YOU TO DO IT. SEE SECTION 4.**</font>

🐼 `pandas` allows you to export your dataframe to a SQLite database using the [pd.to_sql() function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html):

```python

df.to_sql('table_name', con=connection_object)

```

But first, as indicated above, we need to establish a connection to the database.

**Establishing a connection to a database**

In [16]:
# Create a database engine using SQLAlchemy
engine = create_engine('sqlite:///../data/reddit/reddit.db', echo=False, isolation_level="AUTOCOMMIT")

# Why? Read: https://stackoverflow.com/a/71685414/843365
with engine.connect() as conn:
    pass

**Once you have a connection to a database, storing a DataFrame in it is as simple as:**


In [17]:
df.to_sql('top_posts', engine, if_exists='replace', index=False)

1000

If you have a VSCode extension, like [SQLite Viewer](https://marketplace.visualstudio.com/items?itemName=qwtel.sqlite-viewer) you can easily inspect the database.

## 3.3 Overwriting the table

If you made a mistake or changed your mind about the data you stored in the database, you can overwrite the table by setting the `if_exists` parameter to `'replace'`.

For example, I realised that my DataFrame did not store the rank of the post. I can fix the DataFrame in Python and then overwrite the table in the database:

In [18]:
# TODO: It would be better to do this in the function that reads the data, not here.
df['rank'] = range(1, df.shape[0] + 1)

In [19]:
df.to_sql('top_posts', engine, if_exists='replace', index=False)

1000

## 3.4 Reading from a database

If you have a connection to a database, you can read a table from it using the `pd.read_sql()` function:

In [20]:
df_from_db = pd.read_sql('top_posts', engine)

In [21]:
df_from_db.head(4)

,id,title,permalink,post_hint,url,created_utc,ups,upvote_ratio,score,num_comments,is_original_content,is_self,author,author_fullname,content_categories,edited,domain,rank
0,1f6ztk4,How to avoid 1/2-assed data analysis,https://www.reddit.com/r/datascience/comments/...,image,https://i.redd.it/l6zd3zsa9cmd1.jpeg,2024-09-02 07:18:04,3122,0.98,3122,56,False,False,BdR76,t2_sejrw,None,0,i.redd.it,1
1,1feyx6h,Favourite piece of code 🤣,https://www.reddit.com/r/datascience/comments/...,image,https://i.redd.it/jqvbe1ycrcod1.jpeg,2024-09-12 11:06:40,2810,0.98,2810,102,False,False,nobody_undefined,t2_dh9o0mnfv,None,0,i.redd.it,2
2,1dsnbww,"You're not helping, Excel! please STOP HELPING!!!",https://www.reddit.com/r/datascience/comments/...,image,https://i.redd.it/ccxo0ajs3v9d1.jpeg,2024-07-01 08:46:27,1808,0.97,1808,155,False,False,BdR76,t2_sejrw,None,0,i.redd.it,3
3,1afgvv9,Friendly reminder not to work too hard. You'll...,https://www.reddit.com/r/datascience/comments/...,None,https://www.reddit.com/r/datascience/comments/...,2024-01-31 12:37:25,1704,0.93,1704,204,False,True,[deleted],None,None,0,self.datascience,4


# 4. A more principled approach

While `to_sql()` is easy and convenient, it is NOT the most efficient way to store data in a database.

Similar to Pandas, each column of a SQL table has a data type. When you use `to_sql()`, Pandas will try to infer the data type of each column but it will always default to a 'safe' data type. This can be inefficient if you know the data types of your columns could be more specific.

**What are the data types of my DataFrame in Pandas?**

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   1000 non-null   object        
 1   title                1000 non-null   object        
 2   permalink            1000 non-null   object        
 3   post_hint            159 non-null    object        
 4   url                  1000 non-null   object        
 5   created_utc          1000 non-null   datetime64[ns]
 6   ups                  1000 non-null   int64         
 7   upvote_ratio         1000 non-null   float64       
 8   score                1000 non-null   int64         
 9   num_comments         1000 non-null   int64         
 10  is_original_content  1000 non-null   bool          
 11  is_self              1000 non-null   bool          
 12  author               1000 non-null   object        
 13  author_fullname      924 non-null 

You might remember from classes past that the 'object' data type in Pandas is a catch-all for any data type that is not a number or a boolean. Typically, this means that the data type is a string.

In SQL, you can specify the [data type](https://www.w3schools.com/sql/sql_datatypes.asp) of each column. Confusingly, SQLite has its own data types that are similar to other SQL databases but not exactly the same.

<font style="color:#e26a4f;font-weight:bold;">Click [here](https://www.sqlite.org/datatype3.html) to learn more about SQLite-specific data types.</font>

💡 **There are several data types to represent strings.** 

- There is a data type called `TEXT` that can store text of any length. Pandas will always default to `TEXT` when you use `to_sql()`.

- But if you _know_ that your text always have a pre-determined number of characters, you can use `CHAR(N)` where `N` is the number of characters. This will be more efficient in terms of storage.

- If you _know_ that your text can vary in length but you have an upper limit, you can use `VARCHAR(N)` where `N` is the maximum number of characters.

**Similarly, there are several data types to represent numbers, and there are specific data types for dates and times, too.**

## 4.1 Check the limits of my data

Practice using specific data types by checking the limits of your data.

Here is what I suggest you do:

- Check all the integers in your DataFrame and establish the minimum and maximum values. Should I store it as `INT2`, `INT8`, `TINYINT`, `BIGINT`, etc.?

- Don't worry too much about floating-point numbers. Feel free to just use `REAL` or `FLOAT` if you have numbers with decimal points.

- Check all the strings in your DataFrame and establish the maximum length of each string. Should I store it as `CHAR(N)`, `VARCHAR(N)`, or `TEXT`?

<span style="display:block;background-color:rgba(93, 158, 188, 0.1);padding:0.5em;font-size:1.05em;margin-left:0em;margin-bottom:1em;border-radius:0.5em;width:60%">🤔 **Think about it:** Sometimes the decision is not about what is the min and max value in your dataset but what is the **theoretical minimum and maximum**. If you choose a data type that is too narrow and later on you need to add a value that goes beyond the limits of that column, you will get an error.</span>

In [23]:
# My integer columns
int_columns = [col for col in df.columns if df[col].dtype == 'int64']

# My float columns
float_columns = [col for col in df.columns if df[col].dtype == 'float64']

# My text columns (potentially)
text_columns = [col for col in df.columns if df[col].dtype == 'object']

### Checking my integers

Checking the limits of the integers in my DataFrame:

In [24]:
df[int_columns].describe().loc[['min', 'max'],:]

,ups,score,num_comments,rank
min,19.0,19.0,2.0,1.0
max,3122.0,3122.0,641.0,1000.0


Based on this, I decided on the following data types:

- `ups`: SMALLINT (it seems unlikely that this number will ever be larger than 32,767)

- `score`: SMALLINT

- `num_comments`: SMALLINT

- `rank`: UNSIGNED BIG INT (I feel greedy, so I might want to store more than 4 billion posts in the future)

### Checking my strings

In [25]:
df[text_columns].apply(lambda x: x.str.len().describe()).loc[['min', 'max'],:]

,id,title,permalink,post_hint,url,author,author_fullname,content_categories,edited,domain
min,7.0,9.0,64.0,4.0,20.0,4.0,8.0,NaN,NaN,0.0
max,7.0,295.0,105.0,10.0,188.0,20.0,13.0,NaN,NaN,27.0


Based on the above I decided that I will remove the `content_categories` and `edited` columns from my DataFrame, they do not seem to be very useful.

I have also decided on the following data types for the remaining columns:

- `id`: CHAR(6) (I know that Reddit post IDs are **always** 6 characters long)

- `title`: VARCHAR(300) (I don't know the maximum length of a Reddit post title, but it's fairly safe to assume it will be less than 300 characters)

- `permalink`: TEXT (links can get long, so I'd rather not risk cutting them off)

- `post_hint`: VARCHAR(20) (I know that the post_hint column can only have a few values, so I can limit the length of the string)

- `url`: TEXT

- `author`: VARCHAR(50) (I know that Reddit usernames can only have a few values, so I can limit the length of the string)

- `author_fullname`: VARCHAR(100) (I don't know the maximum length of a Reddit author fullname, but it's fairly safe to assume it will be less than 100 characters)

- `domain`: VARCHAR(100) (I'll limit but not too much)

In [26]:
df = df.drop(columns=['content_categories', 'edited'])

## 4.2 SQL's CREATE TABLE statement

It is best to design your table first, using [SQL's own language](https://www.w3schools.com/sql/) and then insert the data into it. This way, you can specify the data type of each column.

Here is an example of how you can create a table in SQLite while specifying the data type of each column:



In [28]:
create_statement = text("""
CREATE TABLE IF NOT EXISTS top_posts (
    id CHAR(6) PRIMARY KEY,
    title VARCHAR(300),
    permalink TEXT,
    post_hint VARCHAR(20),
    url TEXT,
    created_utc DATETIME,
    ups SMALLINT,
    upvote_ratio FLOAT,
    score SMALLINT,
    num_comments SMALLINT,
    is_original_content BOOLEAN,
    is_self BOOLEAN,
    author VARCHAR(50),
    author_fullname VARCHAR(100),
    content_categories TEXT,
    edited BOOLEAN,
    domain VARCHAR(100),
    rank UNSIGNED BIG INT
);
""")

# Now we can create the table

with engine.connect() as conn:
    conn.execute(text('DROP TABLE IF EXISTS top_posts;'))
    conn.execute(create_statement)

## 4.3 Inserting data into an existing table

After the code above, the table will be there but it will be empty. We can now use the `pd.to_sql()` function to insert the data into the table.

In [29]:
# Let's insert the data into the table
df.to_sql('top_posts', engine, if_exists='append', index=False)

1000

## 4.4 Adding a different table

If I decide to store the comments in a different table, I can create a new table for them:

```sql
CREATE TABLE comments (
    id CHAR(6) PRIMARY KEY,
    post_id CHAR(6),
    author VARCHAR(50),
    body TEXT,
    score SMALLINT,
    FOREIGN KEY (post_id) REFERENCES posts(id)
);
```

The [foreign key](https://www.w3schools.com/sql/sql_foreignkey.asp) constraint ensures that the `post_id` in the `comments` table must reference an existing `id` in the `posts` table. This makes merging (joining) the two tables easier.



In [30]:
create_comment_table = text("""
    CREATE TABLE comments (
        id CHAR(6) PRIMARY KEY,
        post_id CHAR(6),
        author VARCHAR(50),
        body TEXT,
        score SMALLINT,
        FOREIGN KEY (post_id) REFERENCES posts(id)
    );
""")

with engine.connect() as conn:
    conn.execute(create_comment_table)

Note that we added a new table to the exact same database file. No need for multiple CSV files or JSON files. Everything is in one place.

# 5. Summary

In this notebook, we have:

- Revisited JSON normalisation using `pd.json_normalize()`

- Discussed how to decide which columns to keep in a DataFrame

- Learned how to store data in a SQLite database using `pd.to_sql()`

- Learned how to create a table in a SQLite database using SQL's `CREATE TABLE` statement

- Learned how to insert data into an existing table in a SQLite database

# 6. What's Next?

Next week we will discuss how to think deeper about the structure of your data, how to link up tables with PRIMARY and FOREIGN KEYS, and how to query data from a database using SQL.

We will also discuss best practices for data visualisation.